(How) does weather influence ammount of bikeleases?

In [21]:
#Data Analysis libraries
import numpy as np
import pandas as pd


In [22]:
#importing needed csv files
stations = pd.read_csv('input/station.csv')
trips = pd.read_csv('input/trip.csv', error_bad_lines=False)
weathers = pd.read_csv('input/weather.csv')

b'Skipping line 50794: expected 12 fields, saw 20\n'


In [23]:
#convert the strings to datetime objects
trips['starttime'] = pd.to_datetime(trips['starttime'])
trips['stoptime'] = pd.to_datetime(trips['stoptime'])


In [24]:
#trim the time of, we only want the date
trips['starttime'] = trips['starttime'].dt.date
trips['stoptime'] = trips['stoptime'].dt.date

In [25]:
#group by trips per day, add trips per day to weather dataframe
tripsperday = trips.groupby(['starttime']).size().reset_index(name='biketrips')
weathers['biketrips'] = tripsperday['biketrips']
weathers

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,biketrips
0,10/13/2014,71,62.0,54,55,51,46,87,68,46,...,29.65,10,10,4,13,4,21,0.00,Rain,818
1,10/14/2014,63,59.0,55,52,51,50,88,78,63,...,29.54,10,9,3,10,5,17,0.11,Rain,982
2,10/15/2014,62,58.0,54,53,50,46,87,77,67,...,29.51,10,9,3,18,7,25,0.45,Rain,626
3,10/16/2014,71,61.0,52,49,46,42,83,61,36,...,29.81,10,10,10,9,4,-,0.00,Rain,790
4,10/17/2014,64,60.0,57,55,51,41,87,72,46,...,29.73,10,10,6,8,3,-,0.14,Rain,588
5,10/18/2014,68,64.0,59,59,57,55,90,83,68,...,29.80,10,8,2,10,4,-,0.31,Rain,798
6,10/19/2014,73,64.0,55,57,55,53,94,74,52,...,29.67,10,10,6,10,3,18,0.00,NaN,1332
7,10/20/2014,66,60.0,55,57,54,50,90,78,67,...,29.69,10,10,5,12,5,-,0.44,Rain,778
8,10/21/2014,64,58.0,55,52,49,46,87,70,58,...,29.74,10,10,6,15,8,21,0.10,Rain,714
9,10/22/2014,60,58.0,57,55,53,48,88,81,67,...,29.56,10,6,2,14,8,22,1.43,Rain,282


In [26]:
#sum the ammount of trips, grouped per unique event
Eventtrips = weathers.groupby('Events').apply(lambda x: x.groupby('Date').biketrips.first().sum()).reset_index(name='tripsperevent')

In [27]:
#refactor the information into a new dataframe to get an overview of how many days each event took place
bla = weathers['Events'].value_counts().reset_index(name='count')
bla = bla.sort_values(by=['index'])
bla.rename(columns={'index': 'Events'}, inplace=True)
bla

,Events,count
1,Fog,16
2,"Fog , Rain",7
3,Fog-Rain,6
0,Rain,287
8,"Rain , Snow",1
5,"Rain , Thunderstorm",3
6,Rain-Snow,2
4,Rain-Thunderstorm,4
7,Snow,2


In [28]:
#Calculate average trips per event per day
Eventtrips['count'] = bla['count']
Eventtrips
Merge = pd.merge(Eventtrips, bla, on='Events'  )
Merge = Merge[['Events','tripsperevent', 'count_y']]
Merge['avg'] = Merge['tripsperevent']/Merge['count_y']
Merge

,Events,tripsperevent,count_y,avg
0,Fog,6528,16,408.000000
1,"Fog , Rain",3302,7,471.714286
2,Fog-Rain,1535,6,255.833333
3,Rain,101934,287,355.170732
4,"Rain , Snow",126,1,126.000000
5,"Rain , Thunderstorm",1355,3,451.666667
6,Rain-Snow,252,2,126.000000
7,Rain-Thunderstorm,1032,4,258.000000
8,Snow,463,2,231.500000


In [29]:
#Calculate trips per day where Event is null (good weather), and add to dataframe
df = pd.DataFrame([['NaN',weathers['biketrips'].sum(),weathers['Events'].isnull().sum(),weathers['biketrips'].sum()/weathers['Events'].isnull().sum()]], columns=('Events','tripsperevent','count_y','avg'))
Merge = Merge.append(df, ignore_index=True)
Merge


,Events,tripsperevent,count_y,avg
0,Fog,6528,16,408.000000
1,"Fog , Rain",3302,7,471.714286
2,Fog-Rain,1535,6,255.833333
3,Rain,101934,287,355.170732
4,"Rain , Snow",126,1,126.000000
5,"Rain , Thunderstorm",1355,3,451.666667
6,Rain-Snow,252,2,126.000000
7,Rain-Thunderstorm,1032,4,258.000000
8,Snow,463,2,231.500000
9,NaN,286857,361,794.617729


conclusion: the weather certainly influences the ammount of leased bicycles. Rain and snow is the the combination that effects the lease the most (126 leases on average). Now that we have seen the absolute values, we want to put this into a nice graph and calculate the correlation

In [31]:
#importing needed csv files again, to reset all changes
station = pd.read_csv('input/station.csv')
trip = pd.read_csv('input/trip.csv', error_bad_lines=False)
weather = pd.read_csv('input/weather.csv')

b'Skipping line 50794: expected 12 fields, saw 20\n'


In [32]:
#refactor columns to needed date and datetime
weather['Date'] = pd.to_datetime(weather['Date'], format="%m/%d/%Y")
weather['year'] = weather['Date'].dt.year
weather['month'] = weather['Date'].dt.month

In [33]:
#calc temp and precipation per month, year. Concatenate them into 1 dataframe
temppermy = weather[['Mean_Temperature_F', 'year', 'month']].groupby(['year','month']).mean()
precpermy = weather[['Precipitation_In', 'year', 'month']].groupby(['year','month']).mean()
result = pd.concat([tripspermy, temppermy, precpermy], axis=1, join='inner')


NameError: name 'tripspermy' is not defined

- Deel door neerslag omdat je verwacht dat het aantal trips groter is bij lage neerslag
- Vermenigvuldig met de temperatuur omdat je verwacht dat het aantal trips grote is bij een hogere temperatuur

In [39]:
#amount of trips / precipitation
result['TripsPrec'] = tripspermy['Amount of trips']/precpermy['Precipitation_In']
#amount of trips / temperature
result["TripsTemp"] = tripspermy['Amount of trips']*temppermy['Mean_Temperature_F']
result

NameError: name 'tripspermy' is not defined

Correlatie tussen neerslag en fietsverhuur (line-graph)

X as = het aantal waarnemingen (de maanden)
Y as = trips/neerslag

In [35]:
result['TripsPrec'].plot()
plt.xlabel('Tijd')
plt.ylabel('Neerslag')

NameError: name 'result' is not defined

Met neerslag is dus wel significant anders, er worden duidelijk minder trips genomen op het moment dat het regent. Maar neerslag is niet de enige definitie van slecht weer. Temperatuur speelt hier ook een grote rol in. We gaan kijken of dit ook invloed heeft en wat die invloed is.

Correlatie tussen temperatuur en fietsverhuur (line-graph)

X as = het aantal waarnemingen (de maanden)
Y as = trips*temperatuur



In [40]:
result[['TripsTemp']].plot()
plt.xlabel('Tijd')
plt.ylabel('Temperatuur')

NameError: name 'result' is not defined

Er is hier dus geen verschil, is namelijk bijna hetzelfde als amount of trips over time (zoals hierboven te zien)



We kunnen dus concluderen dat de neerslag (maar ook het type neerslag) significante invloed heeft op het aantal gehuurde fietsen. Temperatuur daarentegen heeft geen significante invloed. Van alle typen neerslag heeft de combinatie van sneeuw en regen de meeste invloed op het aantal huurders op die dag. 